In [2]:
import json
import pandas as pd

#Add path
df = pd.read_csv('C:\\Users\\utkarsh.dixit\\Downloads\\Datasets\\egov\\EGOV_DATA_2020.csv')
isoCountry = open('C:\\Users\\utkarsh.dixit\\Downloads\\Datasets\\Iso3CountryCodes.json',)
countryDictionary = json.load(isoCountry)

In [12]:
source = "EGOV_DATA_2020"
dataJson = df.to_json(orient="records")
json_object = json.loads(dataJson)

#fetch unique year from the data
uniqueYear=list(set([str(row['Survey Year']) for row in json_object]))

data = {}

#iterate over each datapoint, skip first 2
for dataPoint in list(json_object[0].keys())[2:]:
    data[dataPoint] = {
        "def": '',
        "name": dataPoint,
        "source": 'UNDESA'
    }
    
    subDictData = {"data":{}}
    for year in uniqueYear:
        subDictData["data"].update([(year, {})])
        for row in json_object:
            if str(row['Survey Year']) == year:
                if row['Country Name'].strip() == "Côte d'Ivoire":
                    iso3Name = countryDictionary["Cote d'Ivoire"]
                else:
                    iso3Name = countryDictionary[row['Country Name'].strip()]
                subDictData["data"][year].update(dict({iso3Name:row[dataPoint]}))
        
    subDictData["data"].update([("recentValue", {})])
    subDictData["data"].update([("recentYear", {})])
    data[dataPoint].update(subDictData)
    recentYear = str(max(list(set([row['Survey Year'] for row in json_object]))))
    
    for row in json_object:
        if str(row['Survey Year']) == recentYear:
            if row['Country Name'].strip() == "Côte d'Ivoire":
                iso3NameNew = countryDictionary["Cote d'Ivoire"]
            else:
                iso3NameNew = countryDictionary[row['Country Name'].strip()]
            subDictData["data"]["recentValue"].update(dict({iso3NameNew:row[dataPoint]}))
            subDictData["data"]["recentYear"].update(dict({iso3NameNew:row['Survey Year']}))
    
json.dumps(data)

out_file = open("EgovData.json", "w")     
json.dump(data, out_file, indent = 6) 
out_file.close()
